# Segmenting and Clustering Neighborhoods in Toronto

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe¶

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
w_link= "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
rawpage= requests.get(w_link).text

There are other packages and frameworks, like Scrapy. But Beautiful Soup allows you to parse the HTML in a a beautiful way, so that’s what I’m going to use

In [6]:
import bs4 as bs

In [20]:
xml = bs.BeautifulSoup(rawpage,'xml')
table_body = xml.find('table')

With *table* we retrieved the target info:

In [23]:
data = []
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

In [25]:
type(data)

list

In [126]:
labels = ['PostalCode', 'Borough', 'Neighbourhood']
df = pd.DataFrame.from_records(data, columns=labels)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [128]:
mask=(df["Borough"]!="Not assigned") & (df["Neighbourhood"]=="Not assigned")
df.loc[mask, "Neighbourhood"] = df["Borough"]


In [130]:
df.head()

,PostalCode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Clean **Not assigned** and **None** values

In [131]:
df2=df.drop(df[(df["Borough"]=="Not assigned") | (df["Borough"].isnull())].index)

In [132]:
df3 = df2.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
df3 = df3.reset_index()

In [133]:
df3.shape

(103, 3)

In [134]:
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
